In [5]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
import pandas as pd
from tqdm import tqdm
import json


In [6]:
# .env 파일 로드
load_dotenv()

# LangChain OpenAI 임베딩 객체 생성
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# CSV 파일 불러오기
df = pd.read_csv("hanja.csv")  # 파일 경로를 여기에 입력

In [7]:
df.head()

,main_sound,level,hanja,meaning,radical,strokes,total_strokes
0,가,7급Ⅱ,家,"[[['집'], ['가']]]",宀,7,10
1,가,7급,歌,"[[['노래'], ['가']]]",欠,10,14
2,가,5급Ⅱ,價,"[[['값'], ['가']]]",人,13,15
3,가,5급,加,"[[['더할'], ['가']]]",力,3,5
4,가,5급,可,"[[['옳을'], ['가']]]",口,2,5


In [8]:
# 'hanja'와 'radical' 컬럼만 추출
hanjas = df['hanja'].dropna().unique()
radicals = df['radical'].dropna().unique()

# 중복 제거된 한자 및 부수 리스트
unique_items = list(set(hanjas.tolist() + radicals.tolist()))

In [9]:
print(unique_items[:10])
print(len(unique_items))

['稢', '鼈', '靚', '介', '悌', '揉', '批', '磯', '擎', '訑']
6023


In [10]:
# 임베딩 결과 저장 딕셔너리
embeddings = {}

# ✅ 배치 기반 임베딩 함수
def embed_texts_batch(text_list, embeddings={}, batch_size=100):
    # 중복 요청 방지: 이미 있는 건 건너뜀
    new_texts = [text for text in text_list if text not in embeddings]
    
    for i in tqdm(range(0, len(new_texts), batch_size), desc="임베딩 중"):
        batch = new_texts[i:i+batch_size]
        try:
            # LangChain을 통한 배치 임베딩 생성
            batch_embeddings = embeddings_model.embed_documents(batch)
            
            # 결과를 딕셔너리에 저장
            for text, embedding in zip(batch, batch_embeddings):
                embeddings[text] = embedding
                
        except Exception as e:
            print(f"배치 {i // batch_size + 1} 임베딩 실패: {e}")
   
    return embeddings

# 🧠 임베딩 수행
embeddings = embed_texts_batch(unique_items, embeddings)

임베딩 중: 100%|██████████| 61/61 [01:46<00:00,  1.75s/it]


In [11]:
embeddings_model.model

'text-embedding-3-small'

In [12]:
# 결과 저장 (optional)
with open(f"{str(embeddings_model.model)}.json", "w", encoding="utf-8") as f:
    json.dump(embeddings, f, ensure_ascii=False, indent=2)

print("✅ 임베딩 완료 및 저장됨.")

✅ 임베딩 완료 및 저장됨.


In [13]:
type(embeddings)

dict